# Prophet model (META)
Overall process:
- Time series decomposition
- Prophet model (auto parameter selection) with in-built time series cross validation
- Prophet model (auto parameter selection) with expanding window
- Prophet model (default parameters + multiplicative seasonality + logistic growth) --> to test the fitting time difference compared to auto parameter selection model

Packages:
1. prophet
2. scikit-learn
3. snowflake-snowpark-python
4. pandas
5. numpy 
6. matplotlib
7. statsmodels

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging 
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)

from prophet import Prophet
from prophet.plot import plot_cross_validation_metric
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_percentage_error as MAPE_metrics
from sklearn.model_selection import ParameterGrid

# Prediction timeframe: 14 days
# Training timeframe: 56 days (4 weeks)
TEST_SIZE = 14
TRAIN_SIZE = TEST_SIZE * 4

session = get_active_session()
session.use_database("ml")
session.use_schema("retail_store")

data = session.table("store_2_preprocessed_transactions")
data = data.to_pandas()
data = data[["DATE", "TOTAL_SALES"]]
data["DATE"] = pd.to_datetime(data["DATE"])
data = data.sort_values(by='DATE', ignore_index=True)

- Functions for future use

In [ ]:
def plot_graph(train_values, actual_values, predictions):
    """
    Plot a graph showing train data, actual values and predictions.
    
    The function plots three lines:
    1. Training data values
    2. Actual test values 
    3. Predicted values
    
    The x-axis represents time steps and y-axis represents the values.

    Args:
        train_values: Array of training data values to plot
        actual_values: Array of actual test values to plot
        predictions: Array of predicted values to plot
    """
    x_train = np.linspace(0, len(train_values), len(train_values))
    x = np.linspace(len(train_values), len(train_values) + len(actual_values), len(actual_values))

    plt.plot(x_train, train_values)
    plt.plot(x, actual_values)
    plt.plot(x, predictions)
    plt.legend(["Train Data", "Actual Sales", "Predictions"])
    plt.show()

    return


def calculate_smape(actual_values, predictions):
    """
    Calculate Symmetric Mean Absolute Percentage Error (SMAPE) between actual and predicted values.
    
    Args:
        actual_values: Array of actual values
        predictions: Array of predicted values
        
    Returns:
        float: SMAPE score as a percentage between 0 and 100
    """
    return 100/len(actual_values) * np.sum(2 * np.abs(predictions - actual_values) / (np.abs(actual_values) + np.abs(predictions)))

- Decomposing time series data to better understand the data.

Analysis:
- Trend component looks similar for both seasonality
- Seasonal componnet looks better on multiplicative models
- Residual component of the multiplicative model shows a more evenly distribution of residuals around 0 with fewer outliers

Conclusion:
- Use multiplicative seasonality for all Prophet models

### 1. Additive seasonality

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

add_decomposed_data = seasonal_decompose(
    data["TOTAL_SALES"], 
    model="additive",
    extrapolate_trend="freq", 
    period=7
)

add_decomposed_data.plot().show()

### 2. Multiplicative seasonality

In [ ]:
# Multiplicative decompose sales data

# Transform data as multiplicative data can't take in zero and negative numbers
# Add a constant to all values to remove zeroes and negatives
mul_data = data["TOTAL_SALES"] + abs(data["TOTAL_SALES"].min()) + 1

mul_decomposed_data = seasonal_decompose(
    mul_data, 
    model="multiplicative",
    extrapolate_trend="freq", 
    period=7
)

mul_decomposed_data.plot().show()

### 3. Rename columns and split train test set
- Set floor as 0 to prevent model from predicting negative or 0 sales value

In [ ]:
# Date needs to be renamed as "ds" and sales have to be renamed as "y"
data = data.rename(columns={"DATE": "ds"})
data = data.rename(columns={"TOTAL_SALES": "y"})

# Transform data as prophet performance metrics data can't take in zero and negative numbers
# Change negative values into 0
data["y"] = data["y"] + 1

FLOOR = 0
CAP = data["y"].max()

data["floor"] = 0
data["cap"] = CAP

# Split train test set
test_set = data.iloc[-TEST_SIZE:]
data = data.iloc[:-TEST_SIZE]

### 4. Fit Prophet model using in-built cross validation function.

Best MAPE parameter combination:\
{\
    'changepoint_prior_scale': 0.05, \
    'growth': 'logistic', \
    'seasonality_mode': 'multiplicative', \
    'seasonality_prior_scale': 0.1, \
    'weekly_seasonality': 5\
}

Best SMAPE parameter combination:\
{\
    'changepoint_prior_scale': 0.05, \
    'growth': 'logistic', \
    'seasonality_mode': 'multiplicative', \
    'seasonality_prior_scale': 0.1, \
    'weekly_seasonality': 5\
}

Optimal results:
- MAPE value: 0.0303
- SMAPE value: 3.0943
- Tuning time: 100.513s
- Fitting time: 0.2163s

In [ ]:
# Setup up param grid for testing combinations of parameters
params_grid = {
    'changepoint_prior_scale': [0.005, 0.01, 0.05],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'seasonality_mode': ['multiplicative'],
    'growth': ['logistic'],
    'weekly_seasonality':[5,10,15],
}

# Get the optimal parameters by using time series cross validation
opt_params_mape = None
opt_params_smape = None
lowest_mape = float("inf")
lowest_smape = float("inf")
results = []

start_time = time.time()


# Iterate through parameter combinations to find optimal parameters that minimize MAPE and SMAPE metrics
for params in ParameterGrid(params_grid):
    model_1 = Prophet(**params)
    model_1.fit(data)

    df_cv = cross_validation(
        model_1, 
        initial=f"{TRAIN_SIZE} days", 
        period=f"{TEST_SIZE} days", 
        horizon=f"{TEST_SIZE} days"
    )
    df_performance = performance_metrics(df_cv, metrics=["mape", "smape"])

    # Get and store lowest 14 days prediction mape and rmse value
    mape_value = df_performance[df_performance["horizon"] == f"{TEST_SIZE} days"]["mape"].values[0]
    smape_value = df_performance[df_performance["horizon"] == f"{TEST_SIZE} days"]["smape"].values[0]

    if mape_value < lowest_mape:
        lowest_mape = mape_value
        opt_params_mape = params

    if smape_value < lowest_smape:
        lowest_smape = smape_value
        opt_params_smape = params
    
    results.append({
        "params": params,
        "mape": mape_value,
        "smape": smape_value
    })
    
end_time = time.time()


In [ ]:
print(f"Prophet autotuning time: {end_time - start_time} seconds")

- Plot MAPE & SMAPE values overtime

In [ ]:
# Plot MAPE and SMAPE values from grid search results
fig_1, axs = plt.subplots(1,2)
fig_1.tight_layout()

mape_values = [result["mape"] for result in results]
smape_values = [result["smape"] for result in results]

axs[0].plot(mape_values)
axs[0].set_title("Overall MAPE values")

axs[1].plot(smape_values)
axs[1].set_title("Overall SMAPE values")

plt.show()

- Best parameter combinations



In [ ]:
import json

print(f"Best MAPE parameter combinations: ({json.dumps(opt_params_mape, indent=4)})")
print(f"Lowest MAPE value: {lowest_mape}")

print(f"Best SMAPE parameter combinations: ({json.dumps(opt_params_smape, indent=4)})")
print(f"Lowest SMAPE value: {lowest_smape}")

# Storage of optimal parameters 
opt_params_mape = {'changepoint_prior_scale': 0.05, 'growth': 'logistic', 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.1, 'weekly_seasonality': 5}
opt_params_smape = {'changepoint_prior_scale': 0.05, 'growth': 'logistic', 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.1, 'weekly_seasonality': 5}

opt_params = opt_params_mape

- observe changepoints and trend over time

In [ ]:
# Fit Prophet model with MAPE-optimized parameters
model_mape = Prophet(**opt_params_mape)
model_mape.fit(data)

df_future = model_mape.make_future_dataframe(periods=0, freq="D")
df_future["floor"] = FLOOR
df_future["cap"] = CAP
prediction_mape = model_mape.predict(df_future)

# Plot MAPE model predictions and changepoints
fig_mape = model_mape.plot(prediction_mape)
plot_changepoint = add_changepoints_to_plot(fig_mape.gca(), model_mape, prediction_mape)

# Fit Prophet model with SMAPE-optimized parameters
model_smape = Prophet(**opt_params_smape)
model_smape.fit(data)

df_future = model_smape.make_future_dataframe(periods=0, freq="D")
df_future["floor"] = FLOOR
df_future["cap"] = CAP
prediction_smape = model_smape.predict(df_future)

# Plot SMAPE model predictions and changepoints
fig_smape = model_smape.plot(prediction_smape)
plot_changepoint = add_changepoints_to_plot(fig_smape.gca(), model_smape, prediction_smape)

- Plot MAPE & SMAPE validation error across time horizon

In [ ]:
# Fit Prophet model with optimal parameters for cross validation
model_1 = Prophet(**opt_params)
model_1.fit(data)

# Obtain and plot daily cross validation error
df_cv = cross_validation(
    model_1, 
    initial=f"{TRAIN_SIZE} days", 
    period=f"{TEST_SIZE} days", 
    horizon=f"{TEST_SIZE} days"
)

plot_cross_validation_metric(df_cv, metric = "mape")

In [ ]:
plot_cross_validation_metric(df_cv, metric = "smape")

### 5. Predict last 14 days

In [ ]:
# Train Prophet model on the last 56 days and make predictions for 14 days ahead
train_data = data[-TRAIN_SIZE:]

start_time = time.time()
model = Prophet(**opt_params)
model.fit(train_data)
end_time = time.time()

print(f"Prophet model fitting time: {end_time - start_time} seconds")


df_future = model_1.make_future_dataframe(periods=TEST_SIZE, freq="D")
df_future["floor"] = FLOOR
df_future["cap"] = CAP
prophet_predictions = model.predict(df_future)

predictions = prophet_predictions["yhat"].iloc[-TEST_SIZE:].values
val_data = data["y"].iloc[-TEST_SIZE:]

- Calculate MAPE and SMAPE for last 14 days

In [ ]:
mape = MAPE_metrics(test_set["y"], predictions)
smape = calculate_smape(test_set["y"], predictions)

print(f"MAPE value for last 14 days prediction: {mape}")
print(f"SMAPE value for last 14 days prediction: {smape}")

- Plot prediction vs actual graph

In [ ]:
plot_graph(train_data["y"], test_set["y"], predictions)

### 6. Expanding window model

Optimal results:
- MAPE value: 0.0346
- SMAPE value: 3.4115
- Tuning time: 41.846s
- Fitting time: 0.0626s

In [ ]:
# Expanding window cross-validation with 14-day frequency
# For each window:
# 1. Train Prophet model on data up to current date
# 2. Perform cross-validation and calculate performance metrics
# 3. Store MAPE and SMAPE values for analysis
EXPANDING_WINDOW_FREQ = "14D"

ew_data = data.sort_values("ds")
ew_data["ds"] = pd.to_datetime(ew_data["ds"])

start_date = ew_data["ds"].min() + pd.Timedelta(days=TEST_SIZE*4)
end_date = ew_data["ds"].max() - pd.Timedelta(days=TEST_SIZE)

MAPE_values_ew = []
SMAPE_values_ew = []

start_time = time.time()
for date in pd.date_range(start_date, end_date, freq=EXPANDING_WINDOW_FREQ):
    ew_train = ew_data.loc[ew_data["ds"] < date + pd.offsets.Day(TEST_SIZE+1)]

    # Fit model
    model_3 = Prophet(**opt_params)
    model_3.fit(ew_train)

    # Perform cross validation
    df_cv = cross_validation(
        model_3,
        initial=f"{TRAIN_SIZE} days", 
        period=f"{TEST_SIZE} days", 
        horizon=f"{TEST_SIZE} days"
    )

    df_performance = performance_metrics(df_cv, metrics=["mape", "smape"])

    # Store MAPE & SMAPE values
    mape_value = df_performance[df_performance["horizon"] == f"{TEST_SIZE} days"]["mape"].values[0]
    smape_value = df_performance[df_performance["horizon"] == f"{TEST_SIZE} days"]["smape"].values[0]

    MAPE_values_ew.append(mape_value)
    SMAPE_values_ew.append(smape_value)

end_time = time.time()

In [ ]:
print(f"Prophet EW autotuning time: {end_time - start_time} seconds")


- Plot MAPE & SMAPE values overtime.

In [ ]:
fig_1, axs = plt.subplots(1,2)
fig_1.tight_layout()

axs[0].plot(MAPE_values_ew)
axs[0].set_title("Overall MAPE values (Expanding Window)")

axs[1].plot(SMAPE_values_ew)
axs[1].set_title("Overall SMAPE values (Expanding Window)")

plt.show()

- Predict last 14 days

In [ ]:
# Train Prophet model with optimized parameters and uses entire dataset for training 
# to make predictions for the last 14 days
train_data = data

start_time = time.time()
model = Prophet(**opt_params)
model.fit(train_data)
end_time = time.time()

print(f"Prophet EW model fitting time: {end_time - start_time} seconds")

df_future = model.make_future_dataframe(periods=TEST_SIZE, freq="D")
df_future["floor"] = FLOOR
df_future["cap"] = CAP
prophet_predictions = model.predict(df_future)

predictions = prophet_predictions["yhat"].iloc[-TEST_SIZE:].values
val_data = data["y"].iloc[-TEST_SIZE:]

In [ ]:
mape = MAPE_metrics(test_set["y"], predictions)
smape = calculate_smape(test_set["y"], predictions)

print(f"MAPE value for last 14 days prediction: {mape}")
print(f"SMAPE value for last 14 days prediction: {smape}")

In [ ]:
plot_graph(train_data["y"].iloc[-56:], test_set["y"], predictions)

# 7. Default parameters + multiplicative seasonality + logistic growth
- Test the fitting time difference compared to using paramgrid

Optimal results:
- MAPE value: 0.0426
- SMAPE value: 4.2647
- Tuning time: -
- Fitting time: 0.2943s

In [ ]:
# Train Prophet model with default parameters, multiplicative seasonality and logistic growth
# Make predictions for the last 14 days using the last 14 days of data
train_data = data[-TRAIN_SIZE:]

start_time = time.time()
model = Prophet(
    seasonality_mode="multiplicative",
    growth="logistic"
)
model.fit(train_data)
end_time = time.time()

print(f"Prophet model fitting time: {end_time - start_time} seconds")


df_future = model.make_future_dataframe(periods=TEST_SIZE, freq="D")
df_future["floor"] = FLOOR
df_future["cap"] = CAP
prophet_predictions = model.predict(df_future)

predictions = prophet_predictions["yhat"].iloc[-TEST_SIZE:].values
val_data = data["y"].iloc[-TEST_SIZE:]

In [ ]:
mape = MAPE_metrics(test_set["y"], predictions)
smape = calculate_smape(test_set["y"], predictions)

print(f"MAPE value for last 14 days prediction: {mape}")
print(f"SMAPE value for last 14 days prediction: {smape}")

In [ ]:
plot_graph(train_data["y"], test_set["y"], predictions)

### 8. Test out model performance on z-score filtered dataset

In [ ]:
z_score_data = session.table("store_2_z_score_preprocessed_transactions")
z_score_data = z_score_data.to_pandas()
z_score_data = z_score_data[["DATE", "TOTAL_SALES"]]
z_score_data["DATE"] = pd.to_datetime(z_score_data["DATE"])
z_score_data = z_score_data.sort_values(by='DATE', ignore_index=True)

# Date needs to be renamed as "ds" and sales have to be renamed as "y"
z_score_data = z_score_data.rename(columns={"DATE": "ds"})
z_score_data = z_score_data.rename(columns={"TOTAL_SALES": "y"})

FLOOR = 0
CAP = z_score_data["y"].max()

z_score_data["floor"] = 0
z_score_data["cap"] = CAP

# Split train test set
test_set = z_score_data.iloc[-TEST_SIZE:]
z_score_data = z_score_data.iloc[:-TEST_SIZE]

In [ ]:
# Train Prophet model on z-score filtered data and generate predictions
# Uses logistic growth with multiplicative seasonality to fit the model
# Generates predictions for the last 14 days
train_data = z_score_data[-TRAIN_SIZE:]

start_time = time.time()
model = Prophet(
    seasonality_mode="multiplicative",
    growth="logistic"
)
model.fit(train_data)
end_time = time.time()

print(f"Prophet model fitting time: {end_time - start_time} seconds")


df_future = model.make_future_dataframe(periods=TEST_SIZE, freq="D")
df_future["floor"] = FLOOR
df_future["cap"] = CAP
prophet_predictions = model.predict(df_future)

z_predictions = prophet_predictions["yhat"].iloc[-TEST_SIZE:].values


In [ ]:
mape = MAPE_metrics(test_set["y"], z_predictions)
smape = calculate_smape(test_set["y"], z_predictions)

print(f"MAPE value for last 14 days prediction: {mape}")
print(f"SMAPE value for last 14 days prediction: {smape}")

In [ ]:
plot_graph(train_data["y"], test_set["y"], z_predictions)

In [ ]:
session.close()
